Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchain);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height], "weight" => df2[:weight_c]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean        SD        Naive SE       MCSE      ESS
alpha 154.5979637 0.27701751 0.00438003134 0.0040915060 1000
 beta   0.9049155 0.04241934 0.00067070868 0.0006151621 1000
sigma   5.1052171 0.19165691 0.00303036182 0.0026352408 1000

Quantiles:
          2.5%       25.0%       50.0%       75.0%        97.5%   
alpha 154.0729750 154.4060000 154.600500 154.78700000 155.12905000
 beta   0.8207176   0.8761200   0.905215   0.93365325   0.98689365
sigma   4.7447602   4.9741425   5.098485   5.23349000   5.49582900



Save the chains in a JLS file

In [8]:
serialize("m4.4s.jls", chn)

chn2 = deserialize("m4.4s.jls")

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Union{Missing, Float64}[154.402 0.902884 5.01068; 154.793 0.907699 5.23282; … ; 154.331 0.922036 4.68855; 154.332 0.90536 4.81723]

Union{Missing, Float64}[155.151 0.875339 4.93099; 154.989 0.852154 5.19966; … ; 154.611 0.925659 5.34553; 154.609 0.961952 5.27315]

Union{Missing, Float64}[154.364 0.85803 4.97896; 154.202 0.875691 5.14192; … ; 154.821 0.890121 5.04506; 154.571 0.91854 5.40661]

Union{Missing, Float64}[154.831 0.897318 4.78311; 154.388 0.907696 5.27125; … ; 154.521 0.891388 4.86634; 154.667 0.858675 4.87481]

Should be identical to earlier result

In [9]:
describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean        SD        Naive SE       MCSE      ESS
alpha 154.5979637 0.27701751 0.00438003134 0.0040915060 1000
 beta   0.9049155 0.04241934 0.00067070868 0.0006151621 1000
sigma   5.1052171 0.19165691 0.00303036182 0.0026352408 1000

Quantiles:
          2.5%       25.0%       50.0%       75.0%        97.5%   
alpha 154.0729750 154.4060000 154.600500 154.78700000 155.12905000
 beta   0.8207176   0.8761200   0.905215   0.93365325   0.98689365
sigma   4.7447602   4.9741425   5.098485   5.23349000   5.49582900



End of `m4.4s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*